# **Oracle Benchmarking** : Empiric Network vs ChainLink

Focusing on LUNA crash period : May 6 to May 25

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import json

from src.pipeline import EmpiricLoader

## Empiric Network Data Retrieval

We will use both StarkNet Sequencer & a synced Node running Pathfinder to retrieve data. Not that it is not yet "verifyable" since there's no P2P syncing between StarkNet nodes yet -> Therefore it's unprovable getting the correct timestamp nor the correct information of a TX.

In [2]:
ENLoader = EmpiricLoader()

In [5]:
ENLoader.transactions.to_csv('empiric_tx.csv')

In [8]:
from ctc.protocols import chainlink_utils
feed = '0x91e9331556ed76c9393055719986409e11b56f73'


In [9]:
data = await chainlink_utils.async_get_feed_data(feed)

Exception: no default provider specified for network 1